In [5]:
# MNIST 데이터를 이용한 로지스틱 회귀분석
import tensorflow as tf
import numpy as np

# 0~9까지 숫자의 종류
num_classes = 10
# 28*28
num_features = 784

# 로지스틱 회귀분석의 학습 파라미터
learning_rate =0.01
training_steps = 1000
batch_size = 256
display_step =100

In [6]:
# MNIST 데이터 불러온다
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
# float32형태로 변환
x_train, x_test = np.array(x_train, np.float32), np.array(x_test,np.float32)

# 이미지 포맷을 784픽셀의 1차원 배열로 변환해준다
x_train, x_test = x_train.reshape([-1,num_features]), x_test.reshape([-1,num_features])

# 255로 나눠서 [0,1] 값으로 표준화 시켜준다
x_train ,x_test = x_train / 255., x_test / 255.

In [ ]:
'''
Sklearn에서 제공되는 Logistic Regression 모델 생성 함수 파라미터

penalty : 규제에 사용 된 기준을 지정 (l1, l2, elasticnet, none) – default : l2
dual : 이중 또는 초기 공식
tol : 정밀도
C : 규제 강도
fit_intercept : 모형에 상수항 (절편)이 있는가 없는가를 결정하는 인수 (default : True)
intercept_scaling : 정규화 효과 정도
class_weight : 클래스의 가중치
random_state : 난수 seed 설정
solver : 최적화 문제에 사용하는 알고리즘
max_iter : 계산에 사용할 작업 수
multi_class : 다중 분류 시에 (ovr, multinomial, auto)로 설정
verbose : 동작 과정에 대한 출력 메시지
warm_start : 이전 모델을 초기화로 적합하게 사용할 것인지 여부
n_jobs : 병렬 처리 할 때 사용되는 CPU 코어 수
l1_ratio : L1 규제의 비율(Elastic-Net 믹싱 파라미터 경우에만 사용)
'''

In [8]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=10, max_iter=10000, multi_class='multinomial', solver='lbfgs')
model.fit(x_train, y_train)

LogisticRegression(C=10, max_iter=10000, multi_class='multinomial')

In [14]:
print("공격 당하기 전의 데이터를 이용한 다항 로지스틱 회귀분석의 정확도")
print(model.score(x_train, y_train)*100)

공격 당하기 전의 데이터를 이용한 다항 로지스틱 회귀분석의 정확도
94.24166666666667


In [10]:
# DeepFool 공격 선언해서 위의 데이터 공격시켜준다
from art.attacks.evasion import DeepFool
from art.estimators.classification.scikitlearn import SklearnClassifier

classifier = SklearnClassifier(model=model, clip_values=(0, 1))
attack = DeepFool(classifier)
x_test_adv = attack.generate(x_test)

DeepFool:   0%|          | 0/10000 [00:00<?, ?it/s]

In [15]:
# 로지스틱 회귀분석 모델의 정확도
print("DeepFool 공격 당한 데이터를 이용한 다항 로지스틱 회귀분석의 정확도")
print(model.score(x_test_adv, y_test)*100)

DeepFool 공격 당한 데이터를 이용한 다항 로지스틱 회귀분석의 정확도
7.01
